In [1]:
import os
import sys  
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.externals import joblib
from sklearn.ensemble import BaggingRegressor



delay = 2
pre_time = 48#<=60

location = 'BJ'
f = open('/home/dedekinds/aqi_data_submission_'+location+'.csv')
df =pd.read_csv(f)
total_data = df.values
num_of_variable =[145,174][location == 'BJ']#beiing
test_x = total_data[:,:num_of_variable]

def num_out_val(location):
    if location == 'LD':return 2
    else:return 3
    
def recall_aqi(location,data,model_index):
    model_dir = "/home/dedekinds/A_meo_code/aqi48_75"
    model_name = '_lightgbm_'+location+str(model_index)+'.txt'
    model = joblib.load(os.path.join(model_dir, model_name))
    temp_ans = 0
    for mod in model:
        temp_ans+=mod.predict(data, num_iteration=mod.best_iteration_)
    return (temp_ans/len(model))[0]

ans = np.zeros((1,3))
for i in range(len(total_data)):
    data = test_x[i].reshape(-1,num_of_variable)
    temp_ans = []
    for j in range(delay,delay+pre_time*num_out_val(location)):
        temp_ans.append(recall_aqi(location,data,j))
    temp_ans = np.array(temp_ans).reshape(-1,num_out_val(location))
    if location == 'LD':
        temp_ans = np.column_stack((temp_ans,np.zeros( (np.shape(temp_ans)[0],1) )))
    ans = np.row_stack((ans,temp_ans))
ans = np.delete(ans,[0],axis=0)  
    

#处理预测中可能出现的负数
def remove_nagative(ans):
    location  = np.where(ans<0)
    row = location[0]
    col = location[1]
    for i in range(len(row)):
        if row[i]-1>0 and row[i]+1<len(ans) and ans[row[i]-1][col[i]]>0 and ans[row[i]+1][col[i]]>0:
            ans[row[i]][col[i]] = (  ans[row[i]-1][col[i]]+ans[row[i]+1][col[i]]   )/2
    
    for j in range(len(ans[0])):#处理第一行
        if ans[0][j]<0:
            t = 0
            while True:
                if ans[0+t][j]>0:
                    ans[0][j] = ans[0+t][j]
                    break
                else:
                    t+=1
    
    location = np.where(ans<0)#处理连续的负数
    row = location[0]
    col = location[1]
    for i in range(len(row)):
        t = 0
        while True:
            if ans[row[i]-t][col[i]]>0:
                ans[row[i]][col[i]] = ans[row[i]-t][col[i]]
                break
            else:
                t+=1
    return ans 

ans = remove_nagative(ans) 
#smape(tru, ans)
#smape(tru, ans)
pd_data = pd.DataFrame(ans)
print(pd_data)
model_dir = "result"
model_name = 'result_'+location+'_aqi48_75_ans.csv'
pd_data.to_csv(os.path.join(model_dir, model_name))


/home/dedekinds/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/lightgbm/basic.py:186: UserWarning: Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.")


               0           1           2
0      27.186602  124.152175  127.164277
1      23.609756  127.443044  134.571116
2      20.073961  124.832219  140.766951
3      26.962123  133.160562  135.862122
4      33.901157  129.468327  141.224506
5      58.310711  118.117050  148.692213
6      67.043417  107.880204  145.738803
7      90.978936  102.321131  132.269699
8      98.074339   97.202353  137.347861
9     123.265300   95.796026  124.476105
10    110.464218   86.666239  117.481241
11    114.961009   76.410694  105.872749
12    113.538604   79.242811   96.256894
13    110.599142   70.298017  109.890485
14    103.605737   63.130691  108.522011
15    116.842378   68.440392  120.344790
16     97.828981   69.873688  130.523608
17     84.882732   69.845634  119.187073
18     66.069666   65.290567  102.604927
19     63.557367   78.383931  110.230604
20     50.236441   92.088025  112.929966
21     50.309460   74.576079  126.695261
22     45.409242   71.455013  107.551348
23     49.038793